In [4]:
import pandas as pd

#file_path = "sim_squat_data.csv"
file_path = "../real_data/squat_data.csv"
df = pd.read_csv(file_path)

print(df.head())

   timestamp                   UWB   accel_x   accel_y   accel_z    gyro_x  \
0   0.081282                   NaN -0.004327  0.011410 -0.018091  0.080910   
1   0.081468                   NaN -0.004327  0.011410 -0.018091  0.080910   
2   0.171178  0.6697363257408142 m  0.008159  0.026471 -0.032390  0.146081   
3   0.171270                   NaN  0.008159  0.026471 -0.032390  0.146081   
4   0.265449  0.6814282536506653 m  0.017397  0.039628 -0.047562  0.167058   

     gyro_y    gyro_z updown ExcerciseType  
0 -0.033244  0.000891   Stop         Squat  
1 -0.033244  0.000891   Stop         Squat  
2 -0.054648 -0.011465   Stop         Squat  
3 -0.054648 -0.011465   Stop         Squat  
4 -0.067403 -0.071129   Stop         Squat  


In [5]:
#df = pd.read_csv("real_squat_data.csv")
df = df.drop(columns=['timestamp','ExcerciseType'])

df = df[df['updown'] != 'Stop'].reset_index(drop=True)
df['updown'] = df['updown'].replace({'Up': 1, 'Down': 0})

df['UWB'] = df['UWB'].astype('str').str.replace('m', '').str.strip().astype('float')

df.head()

C:\Users\남현원\AppData\Local\Temp\ipykernel_16808\3444880707.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['updown'] = df['updown'].replace({'Up': 1, 'Down': 0})


,UWB,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,updown
0,NaN,0.051498,-0.027570,-0.028056,-0.312749,0.192752,0.179631,0
1,0.872696,-0.009284,0.012057,0.045943,-0.096476,0.041136,0.112540,0
2,NaN,-0.009284,0.012057,0.045943,-0.096476,0.041136,0.112540,0
3,0.974625,0.071720,-0.021751,0.029509,-0.012830,0.058404,0.079839,0
4,NaN,0.071720,-0.021751,0.029509,-0.012830,0.058404,0.079839,0


In [6]:
from ahrs.filters import Madgwick
import numpy as np
from scipy.spatial.transform import Rotation as R

tmp = df['updown'].shift(1).fillna(0)
start_idxs = [0] + df[(tmp == 1) & (df['updown'] == 0)].index.tolist()

squat_sets = []

madgwick = Madgwick()

for i in range(len(start_idxs)):
    start = start_idxs[i]
    end = start_idxs[i + 1] if i + 1 < len(start_idxs) else len(df)

    squat = df.iloc[start:end].copy()
    quat = np.array([1.0, 0.0, 0.0, 0.0])

    for j in range(len(squat)):
        acc = squat.iloc[j][['accel_x', 'accel_y', 'accel_z']].values
        gyro = squat.iloc[j][['gyro_x', 'gyro_y', 'gyro_z']].values

        quat = madgwick.updateIMU(q=quat, gyr=gyro, acc=acc)
        rot_mat = R.from_quat([quat[1], quat[2], quat[3], quat[0]]).as_matrix()

        acc_world = rot_mat @ acc
        gyro_world = rot_mat @ gyro

        squat.iloc[i, 1:4] = acc_world
        squat.iloc[i, 4:7] = gyro_world

    squat_sets.append(squat)

In [7]:
df = pd.concat(squat_sets, ignore_index=True)
print(df.head())

        UWB   accel_x   accel_y   accel_z    gyro_x    gyro_y    gyro_z  \
0       NaN  0.023971 -0.020188  0.055115 -0.168424  0.045837  0.050409   
1  0.872696 -0.009284  0.012057  0.045943 -0.096476  0.041136  0.112540   
2       NaN -0.009284  0.012057  0.045943 -0.096476  0.041136  0.112540   
3  0.974625  0.071720 -0.021751  0.029509 -0.012830  0.058404  0.079839   
4       NaN  0.071720 -0.021751  0.029509 -0.012830  0.058404  0.079839   

   updown  
0       0  
1       0  
2       0  
3       0  
4       0  


In [8]:
tmp = df['updown'].shift(1).fillna(0)
squat_ends = df[(tmp == 1) & (df['updown'] == 0)].index
print(f"Total squat ends: {len(squat_ends)}")
print(squat_ends)

squat_sets = []

start_index = 0
for end_index in squat_ends:
    one_set = df.iloc[start_index:end_index+1]

    squat_sets.append(one_set)
    print(f"len of oneset:{len(one_set)}")
    #squat_sets.append(one_set)
    print(f"start_index: {start_index}, end_index: {end_index}")
    start_index = end_index + 1


print(f"Total sets: {len(squat_sets)}")

Total squat ends: 67
Index([  95,  184,  272,  369,  450,  528,  602,  671,  791,  883,  982, 1077,
       1172, 1263, 1351, 1444, 1544, 1636, 1719, 1805, 1896, 1983, 2071, 2158,
       2251, 2355, 2431, 2511, 2599, 2691, 2779, 2868, 2955, 3040, 3132, 3214,
       3303, 3390, 3479, 3558, 3638, 3712, 3787, 3867, 3953, 4032, 4115, 4201,
       4296, 4380, 4450, 4517, 4603, 4684, 4763, 4844, 4926, 5005, 5086, 5166,
       5257, 5322, 5396, 5480, 5557, 5650, 5741],
      dtype='int64')
len of oneset:96
start_index: 0, end_index: 95
len of oneset:89
start_index: 96, end_index: 184
len of oneset:88
start_index: 185, end_index: 272
len of oneset:97
start_index: 273, end_index: 369
len of oneset:81
start_index: 370, end_index: 450
len of oneset:78
start_index: 451, end_index: 528
len of oneset:74
start_index: 529, end_index: 602
len of oneset:69
start_index: 603, end_index: 671
len of oneset:120
start_index: 672, end_index: 791
len of oneset:92
start_index: 792, end_index: 883
len of oneset:99

In [9]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

imu_cols = [ 
            'accel_x', 'accel_y', 'accel_z', 
            'gyro_x', 'gyro_y', 'gyro_z']
UWB_cols = ['UWB']

for i, one_set in enumerate(squat_sets):
    one_set['UWB'] = one_set['UWB'].interpolate(method='linear', limit_direction='both')

    imu_scaler = StandardScaler()
    uwb_scaler = MinMaxScaler()

    one_set[imu_cols] = imu_scaler.fit_transform(one_set[imu_cols])
    one_set[UWB_cols] = uwb_scaler.fit_transform(one_set[UWB_cols])

    squat_sets[i] = one_set

C:\Users\남현원\AppData\Local\Temp\ipykernel_16808\3751815522.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_set['UWB'] = one_set['UWB'].interpolate(method='linear', limit_direction='both')
C:\Users\남현원\AppData\Local\Temp\ipykernel_16808\3751815522.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_set[imu_cols] = imu_scaler.fit_transform(one_set[imu_cols])
C:\Users\남현원\AppData\Local\Temp\ipykernel_16808\3751815522.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [10]:
df_normalized = pd.concat(squat_sets, ignore_index=True)

In [11]:
df_normalized.head()

,UWB,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,updown
0,0.723169,0.847622,-0.831826,1.067376,-1.483318,0.816265,0.329999,0
1,0.723169,-0.305767,0.605308,0.871359,-0.961083,0.751001,0.850115,0
2,0.795571,-0.305767,0.605308,0.871359,-0.961083,0.751001,0.850115,0
3,0.867973,2.503732,-0.901515,0.520150,-0.353930,0.990736,0.576363,0
4,0.766610,2.503732,-0.901515,0.520150,-0.353930,0.990736,0.576363,0


In [13]:
df_normalized.to_csv("real_squat_data.csv", index=False)